7.1. Meet the DS Lab Applicants

When you decided to start down the path to becoming a data scientist at WQU, the first thing you did was to register an account with us. Then you took our admissions exam test, and began your data science journey! But not everyone who creates an account takes the admissions exam. Is there a way to improve that completion rate?

In this project, you'll help run an experiment to see if sending a reminder email to applicants can increase the likelihood that they'll complete the admissions exam. This type of experiment is called a hypothesis test or an A/B test.

In this lesson, we'll try to get a better sense of what kind of people sign up for Applied Data Science Lab — where they're from, how old are they, what have they previously studied, and more.

In [ ]:
from pprint import PrettyPrinter

import pandas as pd
import plotly.express as px
from country_converter import CountryConverter
from IPython.display import VimeoVideo
from pymongo import MongoClient

The DS Lab student data is stored in a MongoDB database. So we'll start the lesson by creating a `PrettyPrinter`, and connecting to the right database and collection.

In [ ]:
pp = PrettyPrinter(indent=2)
print("pp type:", type(pp))

Next up, let's connect to the MongoDB server.

Create a client that connects to the database running at localhost on port 27017

In [ ]:
client = MongoClient(host="localhost", port=27017)
print("client type:", type(client))

Let's take a look at the databases that are available to us.

Print a list of the databases available on client.

In [ ]:
pp.pprint(list(client.list_databases()))

Assign the "ds-applicants" collection in the "wqu-abtest" database to the variable name ds_app

In [ ]:
db = client['wqu-abtest']
ds_app = db['ds-applicants']
print("ds_app type:", type(ds_app))

Let's take a look at what we've got. First, let's find out how many applicants are currently in our collection.

Use the count_documents method to see how many documents are in the ds_app collection.

In [ ]:
# Count documents in `ds_app`
n_documents = ds_app.count_documents({})

Use the find_one method to retrieve one document from the ds_app collection and assign it to the variable name result.

In [ ]:
result = ds_app.find_one({})
print("result type:", type(result))
pp.pprint(result)

**THE RETURNED DOCUMENT**

In [ ]:
result type: <class 'dict'>
{ '_id': ObjectId('6525d787953844722c8383f8'),
  'admissionsQuiz': 'incomplete',
  'birthday': datetime.datetime(1998, 4, 29, 0, 0),
  'countryISO2': 'GB',
  'createdAt': datetime.datetime(2022, 5, 13, 15, 2, 44),
  'email': 'terry.hassler28@yahow.com',
  'firstName': 'Terry',
  'gender': 'male',
  'highestDegreeEarned': "Bachelor's degree",
  'lastName': 'Hassler'}

Let's start the analysis. One of the possibilities in each record is the country of origin. We already know WQU is a pretty diverse place, but we can figure out just how diverse it is by seeing where applicants are coming from.

First, we'll perform an aggregation to count countries.
Use the aggregate method to calculate how many applicants there are from each country.

In [ ]:
result = ds_app.aggregate(
    [
        {
            '$group': {'_id': '$countryISO2', 'count': {'$count': {}}}
        }
    ]
)

Put your results from the previous task into a DataFrame named df_nationality. Your DataFrame should have two columns: "country_iso2" and "count". It should be sorted from the smallest to the largest value of "count".

In [ ]:
df_nationality = pd.DataFrame(result).rename({'_id': 'country_iso2'}, axis='columns').sort_values('count')

print("df_nationality type:", type(df_nationality))
print("df_nationality shape", df_nationality.shape)

Now we have the countries, but they're represented using the ISO 3166-1 alpha-2 standard, where each country has a two-letter code. It'll be much easier to interpret our data if we have the full country name, so we'll need to do some data enrichment using country converter library.

Since country_converter is an open-source library, there are several things to think about before we can bring it into our project. The first thing we need to do is figure out if we're even allowed to use the library for the kind of project we're working on by taking a look at the library's license. country_converter has a GNU General Public License, so there are no worries there.

Second, we need to make sure the software is being actively maintained. If the last time anybody changed the library was back in 2014, we're probably going to run into some problems when we try to use it. country_converter's last update is very recent, so we aren't going to have any trouble there either.

Third, we need to see what kinds of quality-control measures are in place. Even if the library was updated five minutes ago and includes a license that gives us permission to do whatever we want, it's going to be entirely useless if it's full of mistakes. Happily, country_converter's testing coverage and build badges look excellent, so we're good to go there as well.

The last thing we need to do is make sure the library will do the things we need it to do by looking at its documentation. country_converter's documentation is very thorough, so if we run into any problems, we'll almost certainly be able to figure out what went wrong.

country_converter looks good across all those dimensions, so let's put it to work!

Instantiate a CountryConverter object named cc, and then use it to add a "country_name" column to the DataFrame df_nationality.

In [ ]:
cc = CountryConverter()
df_nationality["country_name"] = cc.convert(
    df_nationality['country_iso2'], to='name_short'
)

print("df_nationality shape:", df_nationality.shape)
df_nationality.head()

Create a horizontal bar chart of the 10 countries with the largest representation in df_nationality. Be sure to label your x-axis "Frequency [count]", your y-axis "Country", and use the title "DS Applicants by Country".

In [ ]:
# Create horizontal bar chart
fig = px.bar(
    data_frame=df_nationality.tail(10),
    x='count',
    y='country_name',
    orientation='h',
    title='DS Applicants Nationality'
)
# Set axis labels
fig.update_layout(xaxis_title='Frequency [count]', yaxis_title='Country')
fig.show()

Create a "count_pct" column for df_nationality that shows the proportion of applicants from each country.

In [ ]:
df_nationality["count_pct"] = (df_nationality['count'] / df_nationality['count'].sum()) * 100

print("df_nationality shape:", df_nationality.shape)
df_nationality.tail()

Recreate your horizontal bar chart of the 10 countries with the largest representation in df_nationality, this time with the percentages. Be sure to label your x-axis "Frequency [%]", your y-axis "Country", and use the title "DS Applicants by Country".

In [ ]:
# Create horizontal bar chart
fig = px.bar(
    data_frame=df_nationality.tail(10),
    x='count_pct',
    y='country_name',
    orientation='h',
    title='DS Applicants by Country'
)
fig.update_layout(xaxis_title='Frequency [%]', yaxis_title='Country')
# Set axis labels

fig.show()

Add a column named "country_iso3" to df_nationality. It should contain the 3-letter ISO abbreviation for each country in "country_iso2".

In [ ]:
df_nationality["country_iso3"] = cc.convert(df_nationality['country_iso2'], to='ISO3')

print("df_nationality shape:", df_nationality.shape)
df_nationality.head()

Create a function build_nat_choropleth that returns plotly choropleth map showing the "count" of DS applicants in each country in the globe. Be sure to set your projection to "natural earth", and color_continuous_scale to px.colors.sequential.Oranges.

In [ ]:
def build_nat_choropleth():
    fig = px.choropleth(
        data_frame=df_nationality,
        locations='country_iso3',
        color='count',
        projection='natural earth',
        color_continuous_scale=px.colors.sequential.Oranges,
        title='DS Applicants: Nationality'
    )
    return fig


nat_fig = build_nat_choropleth()
print("nat_fig type:", type(nat_fig))
nat_fig.show()

Age
Now that we know where the applicants are from, let's see what else we can learn. For instance, how old are DS Lab applicants? We know the birthday of all our applicants, but we'll need to perform another aggregation to calculate their ages. We'll use the "$birthday" field and the "$$NOW" variable.

Use the aggregate method to calculate the age for each of the applicants in ds_app. Store the results in result.

In [ ]:
result = ds_app.aggregate(
    [
        {
            '$project': {
                'years': {
                          '$dateDiff': {
                              'startDate': '$birthday',
                              'endDate': '$$NOW',
                              'unit': 'year'
                          }
                }
            }
            
        }
    ]
)


Read your result from the previous task into a DataFrame, and create a Series called ages.

In [ ]:
ages = pd.DataFrame(result)['years']

print("ages type:", type(ages))
print("ages shape:", ages.shape)
ages.head()

plot a histogram to show the distribution of ages.

Create function build_age_hist that returns a plotly histogram of ages. Be sure to label your x-axis "Age", your y-axis "Frequency [count]", and use the title "Distribution of DS Applicant Ages".

In [ ]:
def build_age_hist():
    # Create histogram of `ages`
    fig = px.histogram(x=ages, nbins=20, title='DS Applicants: Distribution of Ages')
    # Set axis labels
    fig.update_layout(xaxis_title='Age', yaxis_title='Frequency [counts]')
    return fig


age_fig = build_age_hist()
print("age_fig type:", type(age_fig))
age_fig.show()

Education
Okay, there's one more attribute left for us to explore: educational attainment. Which degrees do our applicants have? First, let's count the number of applicants in each category...

Use the aggregate method to calculate value counts for highest degree earned in ds_app.

In [ ]:
result = ds_app.aggregate(
    [
        {
            '$group': {
                '_id': '$highestDegreeEarned',
                'count': {'$count': {}}
            }
        }
    ]
)


print("result type:", type(result))

Read your result from the previous task into a Series education.

In [ ]:
education = pd.DataFrame(result).rename({'_id': 'highest_degree_earned'}, axis='columns').set_index('highest_degree_earned').squeeze()

print("education type:", type(education))
print("education shape:", education.shape)
education.head()

We need to sort these categories more logically. Since we're talking about the highest level of education our applicants have, we need to sort the categories hierarchically rather than alphabetically or numerically. The order should be: "High School or Baccalaureate", "Some College (1-3 years)", "Bachelor's Degree", "Master's Degree", and "Doctorate (e.g. PhD)". Let's do that with a function.

In [ ]:
def ed_sort(counts):
    """Sort array `counts` from highest to lowest degree earned."""
    degrees = [
        "High School or Baccalaureate",
        "Some College (1-3 years)",
        "Bachelor's degree",
        "Master's degree",
        "Doctorate (e.g. PhD)",
    ]
    mapping = {k: v for v, k in enumerate(degrees)}
    sort_order = [mapping[c] for c in counts]
    return sort_order


education.sort_index(key=ed_sort, inplace=True)
education

Create a function build_ed_bar that returns a plotly horizontal bar chart of education. Be sure to label your x-axis "Frequency [count]", y-axis "Highest Degree Earned", and use the title "DS Applicant Education Levels".

In [ ]:
def build_ed_bar():
    # Create bar chart
    fig = px.bar(
        x=education,
        y=education.index,
        orientation='h',
        title='DS Applicants Education Levels'
    )
    # Add axis labels
    fig.update_layout(xaxis_title='Frequency [counts]', yaxis_title='Highest Degree Earned')

    return fig


ed_fig = build_ed_bar()
print("ed_fig type:", type(ed_fig))
ed_fig.show()